# One-way ANOVA
*By P. Stikker*<br>
https://PeterStatistics.com<br>
https://www.youtube.com/stikpet<br>

## Introduction

The one-way ANOVA is a test that is often used to determine if there are differences in means between different categories. ANOVA is short for ANalysis Of VAriances, since to determine if the means are different an investigation in different variances is used.

This is the test I'd use if I have a nominal and a scale (interval/ratio) variable. 

There a few different ways you can do this test with Python. If you have a pandas dataframe, the easiest method (in my opinion) is by using the Pingouin package or the statsmodels.api. You can simply let it know which fields are involved and basically you're done.

If you have separate lists of scores then the scipy package could be used. 

I'll show an example for all three, and in the appendix how you can even avoid almost entirely packages by going over the formulas involved (it still needs a package for the F-distribution, which would go too far to also do without a package).

Lets get started.

## Example

To show an example, I'll load some data as a pandas dataframe. So I'll need the '<a href="https://pandas.pydata.org">pandas</a>' library:

In [1]:
#!pip install pandas
import pandas as pd

And then load the example data using the <a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html">'read_csv'</a>. 

In [2]:
myDf = pd.read_csv('../../Data/csv/StudentStatistics.csv')
myDf.head()

,RespNr,Location,OAA_ObjCourse,OAA_ObjClass,OAA_CourseExec,OAA_RelActObj,OAA_RelActExa,OAA_RelObjExa,OAA_LearProcAct,OAA_LearProcPrep,...,Over_Grade,Over_Strong,Over_Impr,Gen_Gender,Gen_Age,Gen_SecSchool,Gen_Classes,Gen_NumberSubj,Gen_Time,Comments
0,1.0,Rotterdam,Fully Disagree,Fully Disagree,Fully Disagree,Disagree,Fully Disagree,Fully Disagree,Fully Disagree,Fully Disagree,...,20.0,"None, if there was a teacher that teaches how ...",A better teacher/teaching method,Female,22.0,NaN,NaN,Fully agree,20 < 30,Even when I revise my work I still cannot unde...
1,2.0,Haarlem,Disagree,Disagree,NaN,Fully Disagree,Neither disagree nor agree,Agree,Disagree,Neither disagree nor agree,...,50.0,Blackboard,More motivation! Clearer explanation in class,Male,NaN,The Netherlands,6.0,Disagree,10 < 20,"If the survey is anonymous, there shouldn't be..."
2,3.0,Diemen,Fully agree,Fully agree,Agree,Fully agree,Fully agree,Fully agree,Fully agree,Agree,...,80.0,Notably it has motivated alot about my study c...,NaN,Male,37.0,Africa,7.0,Agree,10 < 20,NaN
3,4.0,Rotterdam,Fully Disagree,Neither disagree nor agree,Disagree,Neither disagree nor agree,Neither disagree nor agree,Fully Disagree,Fully Disagree,Neither disagree nor agree,...,15.0,The clearly layout of every subject eacht week,The explanation of the teacher and motivation,Female,24.0,The Netherlands,6.0,Agree,10 < 20,Practice exams
4,5.0,Haarlem,Disagree,Agree,Fully Disagree,Neither disagree nor agree,Fully agree,Fully agree,Neither disagree nor agree,Fully agree,...,40.0,The online learning material,Classes were just really bad and were very con...,Male,19.0,The Netherlands,7.0,Fully agree,10 < 20,NaN


The example will use as a nominal field the 'Location', and as scale field the 'Over_Grade' (the overall grade the student gave for the course).

## Using Pingouin

The first method to perform the one-way ANOVA discussed is using the '<a href="https://pingouin-stats.org">pingouin</a>' package. We need to import that:

In [3]:
# !pip install pingouin
import pingouin as pg

Now we can use pingouin's '<a href="https://pingouin-stats.org/generated/pingouin.anova.html#pingouin.anova">anova</a>' function. This askes for the *dv* (depending variable) which is our scale variable, the *between* which is the variable with the categories (our nominal variable), and if we want to see the results in a nice dataframe with *detailed* set to True.

In [4]:
aov = pg.anova(dv='Over_Grade', between='Location', data=myDf, detailed=True)
aov

,Source,SS,DF,MS,F,p-unc,np2
0,Location,6645.526991,2,3322.763495,8.0431,0.001032,0.263336
1,Within,18590.389676,45,413.119771,NaN,NaN,NaN


The most important result is probably the 'p-unc'. This is the p-value (also known as significance). It is the chance of an F value as in the sample, or even more extreme, if the assumption about the population would be true. The assumption is that all categories have the same mean in the population. With a p-value of 0.001 this is below the usual threshold of 0.05. We would therefor conclude that the location has a significant influence on the mean score of the 'Over_Grade'.

See more details on significance on my website at https://PeterStatistics.com

Lets see how we could get the same results with statsmodels.api.

## Using statsmodels.api

First we import '<a href="https://www.statsmodels.org/">statsmodels.api</a>'.

In [5]:
# !pip install statsmodels.api
import statsmodels.api as sm

Then load the '<a href="https://www.statsmodels.org/devel/generated/statsmodels.regression.linear_model.OLS.html">ols</a>' (short for ordinary least squares) function:

In [6]:
from statsmodels.formula.api import ols

We then use this function to create a model. As input we use the scale variable, followed by a ~, and then the nominal variable. 

We also immediately fit this model.

In [7]:
model = ols('Over_Grade ~ Location', data=myDf).fit()

Last thing to do is now use the '<a href="https://www.statsmodels.org/devel/generated/statsmodels.stats.anova.anova_lm.html#statsmodels.stats.anova.anova_lm">stats.anova_lm</a>' function to see the results:

In [8]:
aovRes = sm.stats.anova_lm(model, typ=2)
aovRes

,sum_sq,df,F,PR(>F)
Location,6645.526991,2.0,8.0431,0.001032
Residual,18590.389676,45.0,NaN,NaN


The same results as before with pingouin.

Now we can also use scipy...

## Using scipy

To perform a one-way ANOVA with scipy, we need to have the scores in separate lists. First we store the nominal and scale variable separately.

In [9]:
myNom = myDf['Location']
myScale = myDf['Over_Grade']

Then, we create a list of booleans (true/false) that is True for each location:

In [10]:
myCat1 = myNom == 'Diemen'
myCat2 = myNom == 'Haarlem'
myCat3 = myNom == 'Rotterdam'

And finally create a list of each scores per category, using those boolean lists:

In [11]:
myCatScores1 = myScale[myCat1].dropna()
myCatScores2 = myScale[myCat2].dropna()
myCatScores3 = myScale[myCat3].dropna()

To perform the one-way ANOVA we can then import the '<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html">f_oneway</a>' function from the '<a href="https://docs.scipy.org/doc/scipy/reference/stats.html">stats</a>' library in the '<a href="https://www.scipy.org/">scipy</a>' package.

In [12]:
from scipy.stats import f_oneway

To use the function, we simply fill in the different lists with scores:

In [13]:
f_oneway(myCatScores1, myCatScores2, myCatScores3)

F_onewayResult(statistic=8.043099681744273, pvalue=0.0010317196274324434)

Less results, but the F-value and p-value are still the same as before.

However, when reporting the results the degrees of freedom (df) is usually also needed. There are three different ones:

\begin{equation*}
df_{between} = k - 1
\end{equation*}
\begin{equation*}
df_{within} = n - 1
\end{equation*}
\begin{equation*}
df_{total} = n - 1
\end{equation*}

The $k$ is the number of categories, which we can simply get by taking the list of unique scores in the nominal field, using '<a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.unique.html">unique</a>', and then Python's '<a href="https://docs.python.org/3/library/functions.html#len">len</a>' to get the length (i.e. the number of elements).

The $n$ is the total number of scores. We can get this by creating a cross table from the nominal and scale variable, using Pandas '<a href="https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html">crosstab</a>', then sum twice (the first gets the row totals, then the sum of those will be the grand total.

So first we get k and n:

In [14]:
k = len(pd.unique(myNom))
n = pd.crosstab(myNom, myScale).sum().sum()
k, n

(3, 48)

The degrees of freedom are then easily found by filling out the formulas:

In [15]:
dfBetween = k - 1
dfWithin = n - k
dfTotal = n - 1

dfBetween, dfWithin, dfTotal

(2, 45, 47)

The df<sub>between</sub> and df<sub>within</sub> are the ones usually reported. See https://PeterStatistics.com for more details on reporting the results.

In the appendix I'll show how you can avoid almost packages entirely by going over the formulas of the one-way ANOVA. Only for the F-distribution itself a package is still needed then.

## Appendix: The Hard Way

First we create true Python lists of the three lists of scores, and store them in a separate list as one:

In [ ]:
scores1 = list(myCatScores1)
scores2 = list(myCatScores2)
scores3 = list(myCatScores3)

scores = [scores1, scores2, scores3]
print(scores)

We can also have the number of categories using Python's '<a href="https://docs.python.org/3/library/functions.html#len">len</a>':

In [ ]:
k = len(scores)
k

We will need the mean of each category. The formula for the mean is:

\begin{equation*}
\bar{x}_i = \frac{\sum_{j=1}^{n_i} x_{i,j}}{n_i} 
\end{equation*}

Where $\bar{x}_i$ indicates the mean of category i, $x_{i,j}$ the j-th score in category i, and $n_i$ the number of scores in category i.

First the numerator of this fraction. It contains the sum for each category, and lets also keep the number of scores in each category:

In [ ]:
ns = []
sums = []

for i in scores:
    ns.append(len(i))
    sums.append(sum(i))

ns, sums

The means can now simply be calculated:

In [ ]:
means = []

for i in range(k):
    means.append(sums[i]/ns[i])

means

Next we'll need the sum of squares within, which has the scary formula of:

\begin{equation*}
SS_w = \sum_{i=1}^{k} \sum_{j=1}^{n_i} \left( x_{i,j} - \bar{x}_i \right)^2
\end{equation*}

But we actually have all the information we need for this:

In [ ]:
SSw = 0

for i in range(k):
    for j in range(ns[i]):
        SSw = SSw + (scores[i][j] - means[i])**2

SSw

We will also need the mean of all scores:

\begin{equation*}
\bar{x} = \frac{\sum_{i=1}^{n} x_i}{n}
\end{equation*}

where

\begin{equation*}
n = \sum_{i=1}^{k} n_i
\end{equation*}


In [ ]:
nTot = sum(ns)
sumTot = sum(sums)
meanTot = sumTot / nTot

nTot, sumTot, meanTot

Then we can calculate the sum of squares between using:

\begin{equation*}
SS_b = \sum_{i=1}^{k} n_i \times \left( \bar{x}_i - \bar{x} \right)^2
\end{equation*}

Again, we have all we need already so:

In [ ]:
SSb = 0
for i in range(k):
    SSb = SSb + ns[i] * (means[i] - meanTot)**2
    
SSb

Almost there. We need those degrees of freedom. As before:

\begin{equation*}
df_b = k - 1
\end{equation*}


\begin{equation*}
df_w = n - k
\end{equation*}



In [ ]:
dfb = k - 1
dfw = nTot - k

dfb, dfw

The mean square within is then defined as:

\begin{equation*}
MS_w = \frac{SS_w}{df_w}
\end{equation*}


In [ ]:
MSw = SSw / dfw
MSw

And the mean square between as:


\begin{equation*}
MS_b = \frac{SS_b}{df_b}
\end{equation*}



In [ ]:
MSb = SSb / dfb
MSb

Finally the f-value is given by:

\begin{equation*}
F = \frac{MS_b}{MS_w}
\end{equation*}


In [ ]:
F = MSb / MSw
F

To find the corresponding p-value we need the f-distribution. This becomes a bit too complicated to work out so here's where we will need a package. The scipy.stats module has an '<a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f.html">f<a>' function, so lets import that one.

In [ ]:
from scipy.stats import f

Then to find the corresponding p-value we can use the 'sf' function, with the F-value, the degrees of freedom between, and the degrees of freedom within:

In [ ]:
f.sf(F, dfb, dfw)

Et voila. The same as we saw in the other methods.